# Hemodynamic correction




In [71]:
from wfieldtools import *
from wfieldtools.hemocorrection import _hemodynamic_find_coeffs
localdisk = 'd:\\temp_folder'
from scipy.signal import butter,filtfilt

# load decomposed 
U = np.load(pjoin(localdisk,'Ua.npy'))
SVT = np.load(pjoin(localdisk,'SVTa.npy'))
tstart = time.time()
# split channels and subtract the mean to each
SVTa = SVT[:,0::2] 
SVTb = SVT[:,1::2]

# Highpass filter
fs = 30.
highpass= True
nchunks = 1000
if highpass:
    b, a = butter(2,0.1/fs, btype='highpass');
    SVTa = filtfilt(b, a, SVTa, padlen=50)
    SVTb = filtfilt(b, a, SVTb, padlen=50)

# subtract the mean
SVTa = (SVTa.T - np.nanmean(SVTa,axis=1)).T
SVTb = (SVTb.T - np.nanmean(SVTb,axis=1)).T

npix = U.shape[0]
idx = np.array_split(np.arange(0,npix),nchunks)

rcoeffs = runpar(_hemodynamic_find_coeffs,
                 [U[ind,:] for ind in idx],
                 SVTa=SVTa,
                 SVTb=SVTb)

rcoeffs = np.hstack(rcoeffs)
# remove nans and zeros
rcoeffs[np.isnan(rcoeffs)] = 1

T = np.dot(np.linalg.pinv(U),(U.T*rcoeffs).T)
vout = SVTa - np.dot(T.T,SVTb)#np.dot(SVTb.T,T.T).T
# vout -= np.percentile(vout,20,axis=0)

In [1]:
%gui qt5

from  wfieldtools import *
localdisk = 'd:/temp_folder/'
dat_path = glob(pjoin(localdisk,'*.dat'))[0]
dat = mmap_dat(dat_path)
# U = np.load(pjoin(localdisk,'Ua.npy'))
# SVT = np.load(pjoin(localdisk,'SVTa.npy'))
stack = SVDStack(U,Vout,dat.shape[-2:])
del dat


In [2]:
qtgraph_show_svd(stack)